# Autogenerated filters

## Overview

This notebook demonstrates the **Autogenerated Filters** feature of Amazon Bedrock Knowledge Bases, which enhances search capabilities through automatic metadata filtering. By leveraging Amazon Bedrock's foundation models, the Autogenerated Filters feature dynamically interprets user queries and generates appropriate metadata filters based on the defined metadata schema for the Knowledge Base. This improves the quality and relevance of search results without requiring explicit filter specifications.

The Autogenerated Filters feature is enabled through the `implicitFilterConfiguration` parameter within the `vectorSearchConfiguration` of the `retrievalConfiguration` in the `Retrieve` and `RetrieveAndGenerate` API calls. These APIs analyze the user's query, identify relevant metadata attributes based on the specified `implicitFilterConfiguration`, and apply these filters to narrow down the search results.

For example, if a user searches for "marketing reports from last year," the Autogenerated Filters feature can automatically recognize that "last year" refers to a specific time period and apply a filter based on the date metadata field. Similarly, if the query mentions a specific product or department, the system can apply filters based on the corresponding metadata fields.

Let's explore how to implement and utilize Autogenerated Filters with Amazon Bedrock Knowledge Bases for an example use case.



<div class="alert alert-block alert-warning">
<b>Note:</b> if you're executing this notebook in Sagemaker environment, please make sure your Sagemaker execution role has access to `CloudWatchLogsReadOnlyAccess` policy along with other required acess and permissions.
</div>

## 1. Setup
Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

Please ignore any pip dependency error (if you see any while installing libraries)

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade boto3

Note: you may need to restart the kernel to use updated packages.


In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import boto3
print(boto3.__version__)

1.42.2


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [7]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '183631345587')

In [8]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name = 'autogenerated-filters-kb'
knowledge_base_description = "Knowledge Base autogenerated metadata filtering."
bucket_name = f'{knowledge_base_name}-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

# Define data sources
data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2 - Create knowledge bases with fixed chunking strategy
Let's start by creating a [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) to store video games data in csv format. Knowledge Bases allow you to integrate with different vector databases including [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() and [MongoDB Atlas](). For this example, we will integrate the knowledge base with Amazon OpenSearch Serverless. To do so, we will use the helper class `BedrockKnowledgeBase` which will create the knowledge base and all of its pre-requisites:
1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge base
7. Knowledge base data source

We will create a knowledge base using fixed chunking strategy. 

You can chhose different chunking strategies by changing the below parameter values: 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [9]:
knowledge_base_metadata = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source, 
    chunking_strategy = "FIXED_SIZE", 
    suffix = suffix
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['autogenerated-filters-kb-4170340']
buckets_to_check:  ['autogenerated-filters-kb-4170340']
Creating bucket autogenerated-filters-kb-4170340
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_4170340) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '318',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 04 Dec 2025 17:03:43 '
                                                 'GMT',
                                         'x-amzn-requestid': 'bdd5e60c-5fde-44ab-9ca4-a838e2ce2e5f'},
                        'HTTPStatusCode': 200,
             

[2025-12-04 17:05:14,092] p6195 {base.py:258} INFO - PUT https://bl25q32vuaxha33t4dzf.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-4170340 [status:200 request:0.428s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-4170340',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 12, 4, 17, 6, 14, 207654, tzinfo=tzlocal()),
  'description': 'Knowledge Base autogenerated metadata filtering.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:183631345587:knowledge-base/EZZWNAJZ30',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'EZZWNAJZ30',
  'name': 'autogenerated-filters-kb-4170340',
  'roleArn': 'arn:aws:iam::183631345587:role/AmazonBedrockExecutionRoleForKnowledgeBase_4170340',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearc

### 2.1 Download Amazon 2019, 2020, 2021, 2022, & 2023 annual reports and upload it to Amazon S3

Now that we have created the knowledge base, let's populate it with the `sec-10-k reports` dataset to KB. This data is being downloaded from [here](https://ir.aboutamazon.com/annual-reports-proxies-and-shareholder-letters/default.aspx). This data is about Amazon's annual reports, proxies and shareholder letters.

In [10]:
import os

def create_directory(directory_name):    
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    else:
        print(f"Directory '{directory_name}' already exists.")

# Call the function to create the directory
create_directory("sec-10-k")

Directory 'sec-10-k' created successfully.


In [11]:
import requests

def download_file(url, filename):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open the file in write-binary mode
        with open(filename, 'wb') as file:
            # Write the content of the response to the file
            file.write(response.content)
        print(f"File downloaded successfully: {filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

# URL of the files to download
urls = ["https://s2.q4cdn.com/299287126/files/doc_financials/2024/ar/Amazon-com-Inc-2023-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/Amazon-2022-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/Amazon-2021-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Annual-Report.pdf"]


for url in urls:
    # Name for the downloaded file
    filename = url.split('/')[-1]

    # Path to save the downloaded file
    filepath = f"./sec-10-k/{filename}"

    # Call the function to download the file
    download_file(url, filepath)

File downloaded successfully: ./sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2022-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2021-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2020-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/2019-Annual-Report.pdf


Let's upload the annual reports data available in the `sec-10-k` folder to s3.

In [12]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                if not file.startswith('.DS_Store'):
                    file_to_upload = os.path.join(root,file)
                    print(f"uploading file {file_to_upload} to {bucket_name}")
                    s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("sec-10-k", bucket_name)

uploading file sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2022-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2021-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2020-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/2019-Annual-Report.pdf to autogenerated-filters-kb-4170340


#### 2.3 Prepare metadata for ingestion


In [13]:
import json
import re

def generate_matadata(data_dir):
    
    # Loop through all PDF files in the directory
    for filename in os.listdir(data_dir):
        if not filename.startswith('.DS_Store'):
            # Define the metadata dictionary
            metadata ={}
            
            filename= f'{data_dir}/{filename}'
            print(filename)
            
            # Create metadata
            metadata["company"] = "Amazon"
            metadata["ticker"] = "AMZN"
            metadata["year"] = re.search(r'\d+', filename.split('/')[-1]).group(0)

            # Create a JSON object
            json_data = {"metadataAttributes": metadata}

            # print(json_data)

            # Write the JSON object to a file
            with open(f"{filename.replace('.pdf', '.pdf.metadata.json')}", "w") as f:
                json.dump(json_data, f)


In [14]:
data_dir = './sec-10-k'
generate_matadata(data_dir)

./sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf
./sec-10-k/Amazon-2022-Annual-Report.pdf
./sec-10-k/Amazon-2021-Annual-Report.pdf
./sec-10-k/Amazon-2020-Annual-Report.pdf
./sec-10-k/2019-Annual-Report.pdf


In [15]:
# upload metadata file to S3
upload_directory("sec-10-k", bucket_name)

uploading file sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2022-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2021-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2020-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/2019-Annual-Report.pdf to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf.metadata.json to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2022-Annual-Report.pdf.metadata.json to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2021-Annual-Report.pdf.metadata.json to autogenerated-filters-kb-4170340
uploading file sec-10-k/Amazon-2020-Annual-Report.pdf.metadata.json to autogenerated-filters-kb-4170340
uploading file sec-10-k/2019-Annual-Report.pdf.metadata.json to autogenerated-filters-kb-4170340


Now start the ingestion job. Since, we are using the same documents as used for fixed chunking, we are skipping the step to upload documents to s3 bucket. 

In [16]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_metadata.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'ZM8Y3KIFXR',
  'ingestionJobId': 'FCHT42KE2S',
  'knowledgeBaseId': 'EZZWNAJZ30',
  'startedAt': datetime.datetime(2025, 12, 4, 17, 6, 50, 87068, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 5,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 5,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 5},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 12, 4, 17, 7, 41, 895087, tzinfo=tzlocal())}
........................................

Finally we save the Knowledge Base Id to test the solution at a later stage. 

In [17]:
kb_id_metadata = knowledge_base_metadata.get_knowledge_base_id()

'EZZWNAJZ30'


### 2.4 Enable Cloudwatch logs for Debugging our Autogenerated filters

Below we're creating some helper functions to enable the cloudwatch logs for debugging.

#### Helper functions for CloudWatch Logs

The helper functions facilitate the analysis of autogenerated filter generation by querying and processing CloudWatch logs. They provide insights into how user queries are leveraged to generate metadata filters. 

These functions allow you to examine the actual filters being generated, verify their consistency across different user query variations, and ensure they align with your expectations and query logic. If any filters are inconsistent or generated unexpectedly, you can use these functions to troubleshoot and understand the reasons behind it.

In [18]:
import boto3
from botocore.exceptions import ClientError

def create_cloudwatch_log_group_for_bedrock(log_group_name):
    logs_client = boto3.client('logs')
    try:
        logs_client.create_log_group(logGroupName=log_group_name)
        print(f"Successfully created CloudWatch log group: {log_group_name}")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
            print(f"Log group {log_group_name} already exists.")
            return True
        else:
            print(f"Error creating log group: {e}")
            return False

def enable_bedrock_invokemodel_logs(cw_log_group_name):
    bedrock_client = boto3.client('bedrock')
    log_group_name = cw_log_group_name
    
    if not create_cloudwatch_log_group_for_bedrock(log_group_name):
        print("Failed to create or confirm log group. Aborting log enablement.")
        return False

    try:
        get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id_metadata)
        role_arn = get_kb_response['knowledgeBase']['roleArn']
        kb_role_name = role_arn.split('/')[-1]

        sts_client = boto3.client('sts')
        account_id = sts_client.get_caller_identity()["Account"]
        role_name = kb_role_name  
        role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"

        # Get all available model types
        response = bedrock_client.list_foundation_models()
        all_model_types = list(set([model['modelName'] for model in response['modelSummaries']]))

        response = bedrock_client.put_model_invocation_logging_configuration(
            loggingConfig={
                'cloudWatchConfig': {
                    'logGroupName': log_group_name,
                    'roleArn': role_arn
                },
                'textDataDeliveryEnabled': True,
                'imageDataDeliveryEnabled': True,
                'embeddingDataDeliveryEnabled': True,
                'videoDataDeliveryEnabled': True
            }
        )
        
        print(f"Successfully enabled InvokeModel logging for Bedrock with log group: {log_group_name}")
        return True
    except ClientError as e:
        print(f"Error enabling InvokeModel logging: {e}")
        return False

def delete_bedrock_invokemodel_log_group(log_group_name):
    log_group_name = "/aws/bedrock/invokemodel"
    logs_client = boto3.client('logs')
    try:
        # First, disable the logging configuration in Bedrock
        bedrock_client = boto3.client('bedrock')
        bedrock_client.delete_model_invocation_logging_configuration()
        print("Successfully disabled InvokeModel logging for Bedrock")

        # Then, delete the log group
        logs_client.delete_log_group(logGroupName=log_group_name)
        print(f"Successfully deleted CloudWatch log group: {log_group_name}")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print(f"Log group {log_group_name} does not exist.")
            return True
        else:
            print(f"Error deleting log group or disabling logging: {e}")
            return False


# Call the function
cw_log_group_name = "/aws/bedrock/invokemodel"

if enable_bedrock_invokemodel_logs(cw_log_group_name):
        print("Bedrock InvokeModel logging has been set up successfully.")
        log_group_name  = cw_log_group_name
else:
    print("Failed to set up Bedrock InvokeModel logging.")


print("Logroup name: ", log_group_name)

Successfully created CloudWatch log group: /aws/bedrock/invokemodel
Successfully enabled InvokeModel logging for Bedrock with log group: /aws/bedrock/invokemodel
Bedrock InvokeModel logging has been set up successfully.
Logroup name:  /aws/bedrock/invokemodel


In [19]:
import time
from datetime import datetime, timezone
from datetime import timedelta
import boto3
import json

cw_client = boto3.client('logs', region_name=region)

def query_model_invocation_log(query_string):
    # Start query
    startTime=int((datetime.now(timezone.utc) - timedelta(minutes=60)).timestamp())
    start_query_response = cw_client.start_query(
        logGroupName=log_group_name,
        startTime=int((datetime.now(timezone.utc) - timedelta(minutes=60)).timestamp()),
        endTime=int(datetime.now(timezone.utc).timestamp()),
        queryString=query_string,
    )
    query_id = start_query_response['queryId']
    # Wait for the query to complete
    response = None
    while response == None or response['status'] == 'Running':
        print('Waiting for query to complete ...')
        time.sleep(1)
        response = cw_client.get_query_results(
            queryId=query_id
        )

    # Print the results
    print(f"Query status: {response['status']}")
    return response

def print_filter_generation_output(user_query):
    print(user_query)
    # Construct CloudWatch Logs Insights query
    # This query:
    # 1. Selects timestamp and message fields
    # 2. Filters for filter generation task messages
    # 3. Matches the user's query
    # 4. Orders results by most recent first
    response = query_model_invocation_log(
        f"""
        fields @timestamp, @message
        | filter @message like /Your task is to structure the user's query to match the request schema provided below./
        | filter input.inputBodyJson.messages.0.content.0.text like /{user_query}/
        | sort @timestamp desc
        """)
    print(response)
    results = response['results']
    if (len(results) == 0):
        print("No results found")
        return
    
    result= results[0][1]['value']
    result_dict = json.loads(result)
    print(f"Generated filter:")
    filter_gen_output = result_dict['output']['outputBodyJson']['output']['message']['content'][0]['text']
    print(filter_gen_output)

### 2.5 Update Knowledge Bases execution role

In [20]:
# Before using autogenerated filters - update the knowledge base execution IAM role with right permissions

iam = boto3.resource('iam')
client = boto3.client('iam')

def get_attached_policies(role_name):
    response = client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    return attached_policies

# get the knowledge base IAM role name
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id_metadata)
role_arn = get_kb_response['knowledgeBase']['roleArn']
role_name = role_arn.split('/')[-1]

# get attached policies
attached_policies = get_attached_policies(role_name)
attached_policies

def update_kb_execution_role(attached_policies, region_name):
    
    for policy in attached_policies:

        print(policy['PolicyArn'])
        policy_name = policy['PolicyName']
        policy_arn = policy['PolicyArn']

        if 'FoundationModel' in policy_arn:
            print('Updating FoundationModel policy: ',policy_arn)
            policy = iam.Policy(policy_arn)
            version = policy.default_version
            policyJson = version.document
            policyJson['Statement'][0]['Resource'].append('arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'.format(region)) 
            policyJson['Statement'][0]['Resource'].append('arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0'.format(region))  
        
            client.detach_role_policy(RoleName=role_name,
                PolicyArn=policy_arn)
            
            response = client.delete_policy(
                PolicyArn=policy_arn
            )
            print(response)
           
            response = client.create_policy(
            PolicyName= policy_name,
            PolicyDocument=json.dumps(policyJson)
            )
            print(response)
        
        client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

In [21]:
update_kb_execution_role(attached_policies, region)
time.sleep(30)

arn:aws:iam::183631345587:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_4170340
Updating FoundationModel policy:  arn:aws:iam::183631345587:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_4170340
{'ResponseMetadata': {'RequestId': 'e1b08fd3-e276-47b8-80e7-528da48a5e9d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 04 Dec 2025 17:08:23 GMT', 'x-amzn-requestid': 'e1b08fd3-e276-47b8-80e7-528da48a5e9d', 'content-type': 'text/xml', 'content-length': '204'}, 'RetryAttempts': 0}}
{'Policy': {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_4170340', 'PolicyId': 'ANPASVQKH66ZVLKTKXXCS', 'Arn': 'arn:aws:iam::183631345587:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_4170340', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2025, 12, 4, 17, 8, 23, tzinfo=tzlocal()), 'UpdateDate': datetime.datetime(2025, 12, 4, 17, 8, 23, tzinfo=tzlocal(

### 2.6 Query the Knowledge Base with Retrieve and Generate API - with metadata (using Autogenerated filters)

Rather creating filters manually, We'll use auto generated filters by Amazon Bedrock Knowledge Bases.

In [22]:
query = "How many prime members does Amazon have after 2021?"

#### 2.6.1 Test - RetreiveAndGenerate API with Autogenerated filters

Let's first test how the `RetrieveAndGenerate` API processes autogenerated filters with a sample user query. As a refresher, the `RetrieveAndGenerate` API is one of the APIs provided by Amazon Bedrock Knowledge Bases. This API queries the knowledge base to retrieve the desired number of document chunks based on a similarity search. It then integrates these retrieved chunks with a LLM to generate an answer to the user's question.



In the code below, we are configuring the `RetrieveAndGenerate` API to use autogenerated filters based on specific metadata attributes. 

The `implicitFilterConfiguration` is a configuration object that defines the metadata attributes that can be used as filters for the similarity search during the retrieval process of the `RetrieveAndGenerate` API. 

This configuration specifies which metadata fields can potentially be used for filtering the retrieved document chunks. In this case, we included three fields:

1. `year`: A number representing the year the document is about.
2. `company`: A string representing the company name the document describes.
3. `ticker`: A string representing the stock ticker symbol of the company.

Each metadata attribute is defined as an object with the following properties:

- `key`: The name of the metadata attribute.
- `type`: The data type of the metadata attribute (e.g., NUMBER, STRING).
- `description`: A human-readable description of the metadata attribute, including any potential values it may take.

The `implicitFilterConfiguration` also specifies the model ARN for the foundation model used to generate filter expressions relevant to the user's query. In this case, it's set to `anthropic.claude-3-5-sonnet-20240620-v1:0`.

In [23]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_metadata,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":10,
                    # "filter": { "equals": { "key": "x-amz-bedrock-kb-data-source-id", "value": ds_id }},
                    "implicitFilterConfiguration": {
                    "metadataAttributes":[
                        {
                            "key": "year",
                            "type": "NUMBER",
                            "description": "The year in which the document is about."
                        },
                        {
                            "key": "company",
                            "type": "STRING",
                            "description": "The company name the document is describing. Possible values include ['Amazon']"
                        },
                        {
                            "key": "ticker",
                            "type": "STRING",
                            "description": "The ticker name of the company. Possible values include ['AMZN']"
                        }
                    ],
                    "modelArn": "arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0".format(region)
                },
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

Unfortunately, I could not find any information in the provided search results about the number of Amazon Prime members after 2021. The search results appear to contain financial information and excerpts from Amazon shareholder letters, but do not mention specific Prime membership numbers.



Now, let's check the CloudWatch logs for the autogenerated filter generated for the given user query.

The `RetrieveAndGenerate` API, with the autogenerated filter configuration, processes the user query and generates a structured filter expression based on the metadata attributes. This filter expression is represented as a JSON object containing logical operations (`and`, `or`) and comparison statements (`eq`, `gt`, `lt`, etc.). The comparison statements specify the metadata attributes and their corresponding values to filter the documents.

Behind the scenes, the model determines the logical operations and values that constitute the filter expressions. It does this by analyzing the user query and the configured metadata attributes. The model identifies the relevant metadata attributes and their values mentioned in the query, and then constructs the filter expression accordingly, using appropriate logical operations and comparison statements.

In [24]:
# check cloudwatch logs for the implict filter generated
time.sleep(60)
print_filter_generation_output(query)

How many prime members does Amazon have after 2021?
Waiting for query to complete ...
Query status: Complete
{'queryLanguage': 'CWLI', 'results': [], 'statistics': {'recordsMatched': 0.0, 'recordsScanned': 58.0, 'estimatedRecordsSkipped': 0.0, 'bytesScanned': 2587896.0, 'estimatedBytesSkipped': 0.0, 'logGroupsScanned': 1.0}, 'status': 'Complete', 'ResponseMetadata': {'RequestId': '0627505e-e28c-4ca5-b88a-186f2507d4ae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0627505e-e28c-4ca5-b88a-186f2507d4ae', 'content-type': 'application/x-amz-json-1.1', 'content-length': '221', 'date': 'Thu, 04 Dec 2025 17:10:00 GMT'}, 'RetryAttempts': 0}}
No results found


#### 2.6.2 Test - Retrieve API with autogenerated filters

Next, we will test the autogenerated filter configuration with the `Retreive` API, another API provided by Amazon Bedrock Knowledge Bases which converts user queries into embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom workﬂows on top of the semantic search results. The output of the Retrieve API includes the the retrieved text chunks, the location type and URI of the source data, as well as the scores of the retrievals.

As we can see, some of the chunks returned by the `Retrieve` API includes associated metadata that has been filtered based on our autogenerated filters. 

1. Relevant Year: Notice that the chunks have a `year` metadata field set to greater than 2021, matching our query about "Amazon's prime members after 2021".

2. Company Information: The `company` field in the metadata consistently shows "Amazon", which aligns with our query about a specific company .

3. Stock Ticker Information: The `ticker` field consistently shows "AMZN", which aligns with our query about a specific stock.

This demonstrates how the autogenerated filters are effectively narrowing down the search results based on the query's implied criteria, even without explicit filter specifications. The system has interpreted the user query about revenue in 2023 and automatically applied filters to return the most relevant information from the knowledge base.

In [25]:
response_ret_with_implicit_fiters = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_metadata, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":10,
            "implicitFilterConfiguration": {
                    "metadataAttributes":[
                        {
                            "key": "year",
                            "type": "NUMBER",
                            "description": "The year in which the document is about."
                        },
                        {
                            "key": "company",
                            "type": "STRING",
                            "description": "The company name the document is describing. Possible values include ['Amazon']"
                        },
                        {
                            "key": "ticker",
                            "type": "STRING",
                            "description": "The ticker name of the company. Possible values include ['AMZN']"
                        }
                    ],
                    "modelArn": "arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0".format(region)
                },
        } 
    },
    retrievalQuery={
        "text": query
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(retrieve_resp['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret_with_implicit_fiters)

Chunk 1:  , Excite, Netscape, GeoCities, AltaVista, @Home, and Prodigy.Infrastructure During 1997, we worked hard to expand our business infrastructure to support these greatly increased     traffic, sales, and service levels:     • Amazon.com’s employee base grew from 158 to 614, and we significantly strengthened our management team.     • Distribution center capacity grew from 50,000 to 285,000 square feet, including a 70% expansion of our Seattle facilities and the launch of our second distribution center in Delaware in November.     • Inventories rose to over 200,000 titles at year-end, enabling us to improve availability for our customers.     • Our cash and investment balances at year-end were $125 million, thanks to our initial public offering in May 1997 and our $75 million loan, affording us substantial strategic flexibility.     Our Employees The past year’s success is the product of a talented, smart, hard-working group, and I take great pride in     being a part of this tea

In [26]:
# check cloudwatch logs for the implict filter generated
time.sleep(20)
print_filter_generation_output(query)

How many prime members does Amazon have after 2021?
Waiting for query to complete ...
Query status: Complete
{'queryLanguage': 'CWLI', 'results': [], 'statistics': {'recordsMatched': 0.0, 'recordsScanned': 102.0, 'estimatedRecordsSkipped': 0.0, 'bytesScanned': 4582762.0, 'estimatedBytesSkipped': 0.0, 'logGroupsScanned': 1.0}, 'status': 'Complete', 'ResponseMetadata': {'RequestId': '0366804d-b06a-40c2-93c4-b0808de68f0e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0366804d-b06a-40c2-93c4-b0808de68f0e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '222', 'date': 'Thu, 04 Dec 2025 17:10:25 GMT'}, 'RetryAttempts': 0}}
No results found


### 2.7 Clean up
Please make sure to uncomment and run below cells to delete the resources created in this notebook.

In [27]:
# delete local directory
import shutil

dir_path = "sec-10-k" # Replace with the actual path

try:
    shutil.rmtree(dir_path)
    print(f"Directory '{dir_path}' and its contents have been deleted successfully.")
except FileNotFoundError:
    print(f"Directory '{dir_path}' not found.")
except Exception as e:
        print(f"An error occurred: {e}")

Directory 'sec-10-k' and its contents have been deleted successfully.


In [28]:
# Delete log group name
delete_bedrock_invokemodel_log_group(log_group_name)

Successfully disabled InvokeModel logging for Bedrock
Error deleting log group or disabling logging: An error occurred (AccessDeniedException) when calling the DeleteLogGroup operation: User: arn:aws:sts::183631345587:assumed-role/bedrock-workshop-studio-SageMakerExecutionRole-8FXonATgO0I1/SageMaker is not authorized to perform: logs:DeleteLogGroup on resource: arn:aws:logs:us-west-2:183631345587:log-group:/aws/bedrock/invokemodel:log-stream: because no identity-based policy allows the logs:DeleteLogGroup action


False

In [29]:
## Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'R98EME339VHEGJDZ',
  'HostId': 's7AVieRwqhEq8kO7N742xerekV/FiB6rB3GDG3gwXwopA370NbODzNrn+laZJlM227pEhFgWUsQ=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 's7AVieRwqhEq8kO7N742xerekV/FiB6rB3GDG3gwXwopA370NbODzNrn+laZJlM227pEhFgWUsQ=',
   'x-amz-request-id': 'R98EME339VHEGJDZ',
   'date': 'Thu, 04 Dec 2025 17:10:27 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [30]:
print("===============================Knowledge base==============================")
knowledge_base_metadata.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

===============================Knowledge base==============================
Deleted data source ZM8Y3KIFXR
======== Knowledge base and all data sources deleted =========
Bucket autogenerated-filters-kb-4170340 does not exist, skipping deletion
======== S3 bucket deletion process completed =========
Found role AmazonBedrockExecutionRoleForKnowledgeBase_4170340
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_4170340========
 [{'PolicyName': 'AmazonBedrockCloudWatchPolicyForKnowledgeBase_4170340', 'PolicyArn': 'arn:aws:iam::183631345587:policy/AmazonBedrockCloudWatchPolicyForKnowledgeBase_4170340'}, {'PolicyName': 'AmazonBedrockS3PolicyForKnowledgeBase_4170340', 'PolicyArn': 'arn:aws:iam::183631345587:policy/AmazonBedrockS3PolicyForKnowledgeBase_4170340'}, {'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_4170340', 'PolicyArn': 'arn:aws:iam::183631345587:policy/AmazonBedrockOSSPolicyForKnowledgeBase_4170340'}, {'PolicyName': 'AmazonBedrockFoundationModelP